https://www.youtube.com/watch?v=h1BnRBzYjYY

In [1]:
import pandas as pd

data = {"Name": ["Anna", "Bob", "Charlie", "Diana", "Eric"], 
        "Age": [20, 34, 23, None, 33],
        "Gender": ["f", "m", "m", "f", "m"], 
        "Job": ["Programmer", "Writer", "Cook", "Programmer", "Teacher"]}

df = pd.DataFrame(data)

In [2]:
df

,Name,Age,Gender,Job
0,Anna,20.0,f,Programmer
1,Bob,34.0,m,Writer
2,Charlie,23.0,m,Cook
3,Diana,NaN,f,Programmer
4,Eric,33.0,m,Teacher


Preprocessing Pipeline:

* Drop Name Feature
* Impute Ages
* Turn Gender into Binary / Numeric
* One Hot Encode Jobs

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Drop Name Feature
df = df.drop(["Name"], axis=1)

# Impute Ages
imputer = SimpleImputer(strategy="mean")
df['Age'] = imputer.fit_transform(df[['Age']])

# Numeric Gender
gender_dct = {"m": 0, "f": 1}
df['Gender'] = [gender_dct[g] for g in df['Gender']]

# OneHotEncode Jobs
encoder = OneHotEncoder()
matrix = encoder.fit_transform(df[['Job']]).toarray()

column_names = ["Programmer", "Writer", "Cook", "Teacher"]

for i in range(len(matrix.T)):
    df[column_names[i]] = matrix.T[i]
    
df = df.drop(['Job'], axis=1)

In [4]:
df

,Age,Gender,Programmer,Writer,Cook,Teacher
0,20.0,1,0.0,1.0,0.0,0.0
1,34.0,0,0.0,0.0,0.0,1.0
2,23.0,0,1.0,0.0,0.0,0.0
3,27.5,1,0.0,1.0,0.0,0.0
4,33.0,0,0.0,0.0,1.0,0.0


----

In [6]:
df2 = pd.DataFrame(data)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class NameDropper(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        imputer = SimpleImputer(strategy="mean")
        X['Age'] = imputer.fit_transform(X[['Age']])
        return X

In [10]:
class AgeImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(['Name'], axis=1)

In [11]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Numeric Gender
        gender_dct = {"m": 0, "f": 1}
        X['Gender'] = [gender_dct[g] for g in X['Gender']]
        
        # OneHotEncode Job
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Job']]).toarray()
        
        column_names = ["Programmer", "Writer", "Cook", "Teacher"]
        
        for i in range(len(matrix.T)):
            X[column_names[i]] = matrix.T[i]
        
        return X.drop(['Job'], axis=1)

In [15]:
dropper = NameDropper()
imp = AgeImputer()
enc = FeatureEncoder()

enc.fit_transform(imp.fit_transform(dropper.fit_transform(df2)))

,Age,Gender,Programmer,Writer,Cook,Teacher
0,20.0,1,0.0,1.0,0.0,0.0
1,34.0,0,0.0,0.0,0.0,1.0
2,23.0,0,1.0,0.0,0.0,0.0
3,27.5,1,0.0,1.0,0.0,0.0
4,33.0,0,0.0,0.0,1.0,0.0


In [17]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("dropper", NameDropper()), 
    ("imputer", AgeImputer()), 
    ("encoder", FeatureEncoder())
])

pipe.fit_transform(df2)

,Age,Gender,Programmer,Writer,Cook,Teacher
0,20.0,1,0.0,1.0,0.0,0.0
1,34.0,0,0.0,0.0,0.0,1.0
2,23.0,0,1.0,0.0,0.0,0.0
3,27.5,1,0.0,1.0,0.0,0.0
4,33.0,0,0.0,0.0,1.0,0.0


In [20]:
data_ = {"Name": ["Fiona", "Gerald", "Hans", "Isabella", "Jacob"], 
        "Age": [20, 34, None, None, 33],
        "Gender": ["f", "m", "m", "f", "m"], 
        "Job": ["Writer", "Programmer", "Programmer", "Programmer", "Teacher"]}

data2 = pd.DataFrame(data_)

In [22]:
pipe.fit_transform(data2)

,Age,Gender,Programmer,Writer,Cook
0,20.0,1,0.0,0.0,1.0
1,34.0,0,1.0,0.0,0.0
2,29.0,0,1.0,0.0,0.0
3,29.0,1,1.0,0.0,0.0
4,33.0,0,0.0,1.0,0.0
